https://blog.naver.com/woosoung1993/221614302574  
  
1. 가상환경 생성  
`conda create -n geo`  
`conda activate geo`  

2. geopandas 설치  
`conda config --env --add channels conda-forge`  
`conda config --env --set channel_priority strict`  
`conda install python=3 geopandas`

In [2]:
!pip install geopandas

     |████████████████████████████████| 994 kB 5.4 MB/s 
     |████████████████████████████████| 6.3 MB 43.0 MB/s 
     |████████████████████████████████| 15.4 MB 38 kB/s 


In [3]:
import geopandas as gpd # shp 파일을 불러오기 위한 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

import folium
import branca # folium에 legend를 표시하기 위한 라이브러리

# 1. 격자 좌표(1km2) 인구 데이터
http://map.ngii.go.kr/ms/map/NlipMap.do?tabGb=statsMap  
\[국토정보맵\] - \[국토통계지도\] - \[총인구\] - \[서울\] - \[격자\] - \[1km\] - \[2021년 04월\] 데이터

In [23]:
seoul_file = "nlsp_020001001.shp"
seoul = gpd.read_file(seoul_file, encoding='utf8')
seoul

,gid,lbl,val,geometry
0,다사6453,9486.00,9486.0,"POLYGON ((964000.000 1953000.000, 964000.000 1..."
1,다사5651,12646.00,12646.0,"POLYGON ((956000.000 1951000.000, 956000.000 1..."
2,다사6157,30514.00,30514.0,"POLYGON ((961000.000 1957000.000, 961000.000 1..."
3,다사5858,24530.00,24530.0,"POLYGON ((958000.000 1958000.000, 958000.000 1..."
4,다사5347,12016.00,12016.0,"POLYGON ((953000.000 1947000.000, 953000.000 1..."
...,...,...,...,...
705,다사4053,None,NaN,"POLYGON ((940000.000 1953000.000, 940000.000 1..."
706,다사7249,None,NaN,"POLYGON ((972000.000 1949000.000, 972000.000 1..."
707,다사4552,None,NaN,"POLYGON ((945000.000 1952000.000, 945000.000 1..."
708,다사6265,None,NaN,"POLYGON ((962000.000 1965000.000, 962000.000 1..."


# 2. shp 파일 분석, 정리

In [6]:
seoul.describe()

,val
count,602.000000
mean,15864.262458
std,12217.052179
min,0.000000
25%,4600.500000
50%,14722.500000
75%,25358.500000
max,45868.000000


In [7]:
seoul.isnull().sum()

gid           0
lbl         108
val         108
geometry      0
dtype: int64

(1) → (2)  
 ↑ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ↓  
(0) ← (3)  
 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;    longitude &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; latitude  
(0) 127.0923035430468 &nbsp; 37.57567440305103  
(1) 127.0922543915847 &nbsp; 37.58468771825672  
(2) 127.1035804064906 &nbsp; 37.58472629867855  
(3) 127.1036281927530 &nbsp; 37.57571297099648

In [8]:
print(seoul.loc[0, 'geometry'])

POLYGON ((964000 1953000, 964000 1954000, 965000 1954000, 965000 1953000, 964000 1953000))


In [9]:
x, y = seoul.loc[0, 'geometry'].exterior.coords.xy
x, y

(array('d', [964000.0, 964000.0, 965000.0, 965000.0, 964000.0]),
 array('d', [1953000.0, 1954000.0, 1954000.0, 1953000.0, 1953000.0]))

In [10]:
list(seoul.loc[0, 'geometry'].exterior.coords)

[(964000.0, 1953000.0),
 (964000.0, 1954000.0),
 (965000.0, 1954000.0),
 (965000.0, 1953000.0),
 (964000.0, 1953000.0)]

## 2-1. None, NaN 값을 0으로 replace
산 지역은 0으로 바꾸는 게 맞는 것 같은데 아닌 지역도 있어서.. **평균으로 해야하는지 생각할 필요 있음**

In [12]:
seoul.describe()

,val
count,710.000000
mean,13451.107042
std,12610.549187
min,0.000000
25%,432.500000
50%,10700.000000
75%,23676.000000
max,45868.000000


## 2-2. geometry의 POLYGON를 geographic coordinate로 변환

https://geopandas.readthedocs.io/en/latest/gallery/polygon_plotting_with_folium.html

In [13]:
seoul.crs

<Projected CRS: EPSG:5179>
Name: Korea 2000 / Unified CS
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Republic of Korea (South Korea) - onshore and offshore.
- bounds: (122.71, 28.6, 134.28, 40.27)
Coordinate Operation:
- name: Korea Unified Belt
- method: Transverse Mercator
Datum: Geocentric datum of Korea
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [24]:
# Use WGS 84 (epsg:4326) as the geographic coordinate system
seoul = seoul.to_crs(epsg=4326)
# seoul = seoul.to_crs({'init': 'epsg:4326'})
print(seoul.crs)
seoul.head()

epsg:4326


,gid,lbl,val,geometry
0,다사6453,9486.00,9486.0,"POLYGON ((127.09230 37.57567, 127.09225 37.584..."
1,다사5651,12646.00,12646.0,"POLYGON ((127.00183 37.55730, 127.00177 37.566..."
2,다사6157,30514.00,30514.0,"POLYGON ((127.05812 37.61161, 127.05806 37.620..."
3,다사5858,24530.00,24530.0,"POLYGON ((127.02407 37.62049, 127.02401 37.629..."
4,다사5347,12016.00,12016.0,"POLYGON ((126.96812 37.52110, 126.96805 37.530..."


In [29]:
seoul.replace([None, np.nan], 0, inplace=True)

# 3. 각 좌표별 인구수 지도에 표시

In [25]:
lat_mean, lon_mean = (37.555774937819336, 126.9955114682439)

In [26]:
m = folium.Map([lat_mean, lon_mean], zoom_start = 12)

In [27]:
max_val = max(seoul['val'])

In [30]:
for _, r in seoul.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    
    color = plt.cm.OrRd(r['val']/max_val)
    color = mpl.colors.to_hex(color)
    
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: {'fillColor': 'orange'})
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })
    folium.Popup(str(int(r['val']))).add_to(geo_j)
    geo_j.add_to(m)

In [31]:
colormap = branca.colormap.linear.OrRd_06.scale(0, max_val)
colormap = colormap.to_step(index=np.linspace(0, max_val, 1000))
colormap.caption = 'AEDs'
colormap.add_to(m)
m

# 4. AED 데이터

## 4-1. AED 데이터 불러오기

In [ ]:
pd_geo_coordi = pd.read_csv('AED_Seoul_coord.csv', engine='python', encoding='CP949')
np_geo_coordi = np.array(pd_geo_coordi.loc[:, ['위도', '경도']]).astype(float)

In [ ]:
lat = np_geo_coordi[:, 0]
lon = np_geo_coordi[:, 1]
lat_mean, lon_mean = np.nanmean(np_geo_coordi, axis = 0)

lat_mean, lon_mean

## 4-2. 각 좌표별 AED 개수

In [ ]:
popups = []
regional_counts = []

for polygon in seoul['geometry']:
    upper_right = list(polygon.exterior.coords)[2]
    lower_left = list(polygon.exterior.coords)[0]

    mask = (
        (pd_geo_coordi.위도 <= upper_right[1]) & (pd_geo_coordi.위도 > lower_left[1]) &
        (pd_geo_coordi.경도 <= upper_right[0]) & (pd_geo_coordi.경도 > lower_left[0])
           )

    region_aeds = len(pd_geo_coordi[mask])
    regional_counts.append(region_aeds)
    popup = folium.Popup(str(region_aeds))
    popups.append(popup)

most_region = max(regional_counts)

## 4-3. 지도에 표시

In [ ]:
m_aed = folium.Map([lat_mean, lon_mean], zoom_start = 12)

In [ ]:
for i, r in seoul.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    
    color = plt.cm.OrRd(regional_counts[i]/most_region)
    color = mpl.colors.to_hex(color)
    
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })
    popups[i].add_to(geo_j)
    geo_j.add_to(m_aed)

In [ ]:
colormap = branca.colormap.linear.OrRd_06.scale(0, most_region)
colormap = colormap.to_step(index=np.linspace(0, most_region, 1000))
colormap.caption = 'AEDs'
colormap.add_to(m_aed)
m_aed

# 5. AED 수 / 인구수